[![Open In Colab](./colab-badge.png)](https://colab.research.google.com/github/subhacom/moose-notebooks/blob/main/Multi-compartmental_neuron_model.ipynb) [![Binder](./binder_logo.png)](https://mybinder.org/v2/gh/subhacom/moose-notebooks/HEAD)

In [ ]:
# Only required on colab!
!pip install pymoose

# Resting membrane potential

All cells have a membrane separating it from the outside environment. As life evolved in the oceans, cellular functions utilize salts dissolved in water (i.e., ions). However, the cell membrane keeps the ion concentrations in the cytoplasm different from the external medium. Moreover, every living cell maintains an a voltage inside the cell membrane relative to the outside. This is called the *membrane potential*.

The cell membrane is semipermeable. When there is a difference in the concentration of an ion across such a membrane, the ions tend to flow from high concentration to low concentration, or down its chemical gradient. The ions being charged particles, they also experience electrical force due to the voltage difference across the membrane.

## Nernst equation
In the "resting" state of a cell, there is an equilibrium so that no net current flows across the cell membrane. The net flow of a given ion (ionic current) is zero when the forces due to electrical gradient and concentration gradient balance out each other. From basic laws of physics one can show that the equilibrium potential for the ion is

$E_{i} = - \frac{RT}{zF} ln \frac{[C]_{in}}{[C]_{out}}$

Here $R$ is the universal gas constant, $T$ the absolute temperature, $F$ Faraday's constant, $z$ the valence of the ion under consideration, $[C]_{in}$ the concentration of the ion inside the cell membrane, and $[C]_{out}$ its concentration in the extrenal solution.

This equation is called the Nernst equation.

## Computing equilibrium potential
While it is relatively easy to compute the equilibrium potential of a given ion using Nernst equation, MOOSE provides a class for doing this computation dynamically (what if the concentration of an ion changes over time?)


In [3]:
import moose as mu


nernst = mu.Nernst('nernst')

A typical mammalian cell has internal K+ concentration of 140 mM and external K+ concentration of 5 mM. Assuming a body temperature of 37 degree C we can set the corresponding fields of the `Nernst` object.

In [7]:
nernst.Temperature = 273.15 + 37.0   # absolute temperature (37 Celcius)
nernst.Cin = 140e-3  # inside-concentration of K+: 140 mM
nernst.Cout = 5e-3   # outside-concentration of K+: 5 mM
nernst.valence = 1   # valence of K+: 1

Setting the values updates the reversal/equilibrium potential right away

In [16]:
print(nernst.E)

-0.08905675664690106


You can verify the result directly in Python

In [12]:
import numpy as np
R = 8.314
T = 273.15 + 37
z = 1
F = 96485.33
Cin = 140.0
Cout = 5.0

ek = - R * T/(z * F) * np.log(Cin/Cout)
print(ek)

-0.08905374075519665


Both give about -89 mV (-0.089 V) as the reversal potential for K+ ion

### Exercise

1. For frog muscle, [Conway 1957](https://doi.org/10.1152/physrev.1957.37.1.84) measured the internal concentration of Cl- ion to be 1.5 mM and the external concentration being 77.5 mM. Assuming the muscle is kept at room temperature (20 degree C), what would be the reversal potential for Cl- in frog muscle?

In [13]:
# Your code here

## Goldman-Hodgkin-Katz (GHK) equation

The [GHK voltage equation](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5688357/) allows us to compute the resting membrane potential when multiple ionic species are distributed inside and outside the cell membrane. If the ions are $Na^+$, $K^{+}$ and $Cl^{-}$, the the membrane potential at equilibrium can be calculated as

$Vm = \frac{RT}{F} \times ln(\frac{P_{K} [K^{+}]_{out} + P_{Na} [Na^{+}]_{out} + P_{Cl} [Cl^{-}]_{in}}{P_{K} [K^{+}]_{in} + P_{Na} [Na^{+}]_{in} + P_{Cl} [Cl^{-}]_{out}})$

Where $P_{C}$ indicates the permeability of the membrane to ion species $C$, $[C]_{out}$ is its concentration outside the membrane, and $C_{in}$ is its concentration inside the membrane. Notice that in the numerator we have the external concentrations of the cations and the internal concentration of the anions, and the opposite in the denominator.

Also be aware that the derivation of the GHK equation assumes the applicability of NPE. Thus the electric field across the membrane should be constant, and there should be no complicated energy barriers in the path of the ions (ions pass through pores made of proteins, and these can have distribution of electron density that creates an energy barrier for the ion).



In [16]:
def GHK_V(K_in, K_out, Na_in, Na_out, Cl_in, Cl_out, P_K, P_Na, P_Cl, T):
    """Compute the resting membrane voltage using GHK voltage equation
    
    """
    num = P_K * K_out + P_Na * Na_out + P_Cl * Cl_in
    den = P_K * K_in + P_Na * Na_in + P_Cl * Cl_out
    return (R * T / F) * np.log(num / den)
    

We can verify it for the example in [Johnston and Wu](https://mitpress.mit.edu/9780262100533/foundations-of-cellular-neurophysiology/)

- K_out = 10, K_in = 400, Na_out = 460, Na_in = 50, Cl_in = 40, Cl_out = 540
- P_K : P_Na : P_Cl = 1: 0.03 : 0.1
- Temperature 20 degree C

Note that although the concentrations are in mM, not in SI, the multipliers cancel out. Similarly, we only need the ratios of the permeabilities as they appear in both the numerator and the denominator.

In [22]:
print(GHK_V(400, 10, 50, 460, 40, 540, 1.0, 0.03, 0.1, 273.15 + 20))

-0.07063690565561223


### Exercise

In a mammalian cell the concentrations of the ions are as follows (in mM):

| Ion |  Inside | Outside 
|-----|---------|--------- 
| $K^{+}$ | 140 |  5
| $Na^{+}$ | 14 | 140
| $Cl^{-}$ | 4  | 100

Assuming permeability of $Na^{+}$ and $Cl^{-}$ to be 0.05 and 0.45 relative to $K^{+}$ (i.e., $P_{K}: P_{Na} : P_{Cl} = 1 : 0.05 : 0.45$), and a temperature of 37 $\degree$ C, compute the resting voltage of the cell using GHK equation.